<a href="https://colab.research.google.com/github/patrick26-Developer/Scatterplot-Graph/blob/main/fcc_book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2025-04-15 09:11:13--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.2’

book-crossings.zip. 100%[===================>]  24.88M  77.0MB/s    in 0.3s    

2025-04-15 09:11:13 (77.0 MB/s) - ‘book-crossings.zip.2’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
# add your code here - consider creating a new cell for each section of code
# Fusionner les notes avec les titres de livres
df = df_ratings.merge(df_books, on='isbn')

# Filtrer les utilisateurs avec au moins 200 évaluations
user_counts = df['user'].value_counts()
df = df[df['user'].isin(user_counts[user_counts >= 200].index)]

# Filtrer les livres avec au moins 100 évaluations
book_counts = df['title'].value_counts()
df = df[df['title'].isin(book_counts[book_counts >= 100].index)]

# Créer la matrice utilisateur-livre
user_book_matrix = df.pivot_table(index='user', columns='title', values='rating')

# Remplacer les valeurs manquantes par 0
user_book_matrix.fillna(0, inplace=True)

# Transposer pour avoir les livres en lignes
book_user_matrix = user_book_matrix.T

# Conversion en matrice creuse pour efficacité mémoire
book_user_sparse = csr_matrix(book_user_matrix.values)

# Entraîner le modèle KNN
model = NearestNeighbors(metric='cosine', algorithm='brute')
model.fit(book_user_sparse)

# Liste des titres de livres pour retrouver les indices plus tard
book_titles = list(book_user_matrix.index)


In [ ]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  if book not in book_titles:
    return [book, []]

  # Obtenir l’index du livre dans la matrice
  book_idx = book_titles.index(book)

  # Trouver les 6 plus proches voisins (y compris lui-même)
  distances, indices = model.kneighbors(book_user_matrix.iloc[book_idx, :].values.reshape(1, -1), n_neighbors=6)

  # Construire la réponse : ignorer le premier (le livre lui-même)
  recommended_books = []
  for i in range(1, len(indices[0])):
    title = book_titles[indices[0][i]]
    dist = np.float32(distances[0][i])
    recommended_books.append([title, dist])

  return [book, recommended_books]


In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()
